In [55]:
#TODO: решить вопрос, должны ли экспериментаторские файлы быть "нормализованы"(нужно ли вычитать контроль?) - иначе
#мы находим пики там, где они не были найдены(Типа MACS не нашел пиков)!
#Вместо предыдущей свистопляски просто считаем кол-во ридов в какждом пике
#(никаких вычитаний контроля и прочего, мы использовали контроль, когда считали пики в каждом образце)
#bedtools coverage -a A.bed -b B.bed, где A это пики, а B это эксперимент
import pandas as pd
import os
from pybedtools import BedTool
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
from multiprocessing import Pool

In [23]:
targets = [("H3K4me1", "_narrow"), ("H3K4me2", "_narrow"), ("H3K4me3", "_narrow"), ("H3K79me2", ""), 
           ("H3K9ac", "_narrow"), ("H3K9me3", ""), ("H4K20me1", "")]

In [56]:
#Список путей к первой реплкие каждого биосемпла
def getFiles(target):
    files = []
    for dirpath, dirnames, filenames in os.walk("/data/mazurovev/" + target[0] + "/"):
        for filename in sorted([f for f in filenames if "_" not in f])[:1]:
            #Делаем путем к файлу с пиками
            files.append(dirpath + "/" + filename)
    
    return sorted(files)

In [44]:
getFiles(("H3K27me3", ""))

['/data/mazurovev/H3K27me3/CL:0000062/ENCFF318IMH.bam',
 '/data/mazurovev/H3K27me3/CL:0000103/ENCFF122QYC.bed',
 '/data/mazurovev/H3K27me3/CL:0000127/ENCFF442ZRV.bed',
 '/data/mazurovev/H3K27me3/CL:0000182/ENCFF208KVX.bed',
 '/data/mazurovev/H3K27me3/CL:0000192/ENCFF327WQJ.bed',
 '/data/mazurovev/H3K27me3/CL:0000515/ENCFF232HUV.bed',
 '/data/mazurovev/H3K27me3/CL:0000746/ENCFF083APJ.bed',
 '/data/mazurovev/H3K27me3/CL:0000842/ENCFF193OFZ.bed',
 '/data/mazurovev/H3K27me3/CL:0002327/ENCFF015OVW.bed',
 '/data/mazurovev/H3K27me3/CL:0002328/ENCFF425HDW.bed',
 '/data/mazurovev/H3K27me3/CL:0002372/ENCFF287IGX.bed',
 '/data/mazurovev/H3K27me3/CL:0002518/ENCFF575FWP.bed',
 '/data/mazurovev/H3K27me3/CL:0002553/ENCFF535LEC.bed',
 '/data/mazurovev/H3K27me3/CL:1001606/ENCFF020YIZ.bed',
 '/data/mazurovev/H3K27me3/EFO:0001187/ENCFF030HAG.bed',
 '/data/mazurovev/H3K27me3/EFO:0001196/ENCFF825OWT.bed',
 '/data/mazurovev/H3K27me3/EFO:0002067/ENCFF190OWE.bed',
 '/data/mazurovev/H3K27me3/EFO:0002074/ENCFF1

In [57]:
def makeCoverage(bed_path):
    exp = BedTool(bed_path)
    print("/".join(bed_path.split("/")[-2:]))
    return ("/".join(bed_path.split("/")[-2:]), peaks.coverage(a=peaks.fn, b=exp.fn).to_dataframe()[['score', 'name']])

In [58]:
def makeSignalMatrix(target, output_file):
    
    pool = Pool(processes=3)
    data = pool.map(makeCoverage, getFiles(target))

    pool.close()
    pool.join()
    
    for_data_frame = {}
    for n, d in data:
        for_data_frame[n] = d['score']
        
    matrix = pd.DataFrame(for_data_frame)
    
    matrix.index = ["peak_" + str(i) for i in matrix.index]
    
    matrix.to_csv("../all_marks/" + target[0] + "/" + output_file, sep="\t")

In [60]:
#кустарный способ подсчета пиков
peaks = BedTool("../all_marks/" + "H3K27me3" + "/merged_narrow_2_peaks_first_in_biosample.bed")
df = makeSignalMatrix(("H3K27me3", ""), "narrow_2_peaks_signal_matrix.csv")

CL:0000515/ENCFF232HUV.bed
CL:0002372/ENCFF287IGX.bed
CL:0000062/ENCFF318IMH.bam


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


CL:0000746/ENCFF083APJ.bed


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


CL:0000103/ENCFF122QYC.bed


...

/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


UBERON:0008952/ENCFF316LIQ.bed


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


UBERON:0011907/ENCFF144WWG.bed


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


In [26]:
#macs2 официальный способ
peaks = BedTool("../all_marks/" + "H3K27me3" + "/merged_narrow_peaks_first_in_biosample.bed")
df = makeSignalMatrix(("H3K27me3", ""), "narrow_peaks_signal_matrix.csv")

CL:0002372/ENCFF287IGX_peaks.narrowPeak
CL:0000062/ENCFF318IMH_peaks.narrowPeak
CL:0000515/ENCFF232HUV_peaks.narrowPeak


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


CL:0000746/ENCFF083APJ_peaks.narrowPeak
CL:0000103/ENCFF122QYC_peaks.narrowPeak


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


CL:0002518/ENCFF575FWP_peaks.narrowPeak


...

/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


UBERON:0008952/ENCFF316LIQ_peaks.narrowPeak


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


UBERON:0011907/ENCFF144WWG_peaks.narrowPeak


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


In [48]:
#macs2 официальный способ - широкие пики
peaks = BedTool("../all_marks/" + "H3K27me3" + "/merged_broad_2_peaks_first_in_biosample.bed")
df = makeSignalMatrix(("H3K27me3", ""), "broad_2_peaks_signal_matrix.csv")

CL:0002372/ENCFF287IGX.bed
CL:0000515/ENCFF232HUV.bed
CL:0000062/ENCFF318IMH.bam


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


CL:0000746/ENCFF083APJ.bed


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


CL:0000103/ENCFF122QYC.bed


...

/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


UBERON:0008952/ENCFF316LIQ.bed


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


UBERON:0011907/ENCFF144WWG.bed


/usr/local/lib/python3.6/dist-packages/pybedtools/bedtool.py:3292: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, names=_names, header=None, *args, **kwargs)


In [10]:
for target in targets:
    print("Make Coverage in Peaks for " + target[0] + "...")
    peaks = BedTool("../all_marks/" + target[0] + "/merged_peaks_first_in_biosample.bed")
    df = makeSignalMatrix(target, "peaks.bed", "peaks_signal_matrix.csv")

CL:0000515/ENCFF190SSF.bed
CL:0002518/ENCFF130UJA.bed
CL:0000062/ENCFF141KCU.bed
CL:0002551/ENCFF008XJN.bed
CL:0002553/ENCFF606TSX.bed
CL:0000746/ENCFF063WTW.bed
EFO:0001187/ENCFF130PYS.bed
CL:0000103/ENCFF258MNP.bed
CL:0002327/ENCFF083BCW.bed
EFO:0001196/ENCFF433YUN.bed
CL:0002328/ENCFF743JIC.bed
EFO:0002067/ENCFF639PLN.bed
EFO:0002074/ENCFF524FTE.bed
CL:0002372/ENCFF474VYS.bed
EFO:0005723/ENCFF558RXY.bed
CL:0000127/ENCFF264MJR.bed
CL:0000182/ENCFF657BPE.bed
EFO:0005904/ENCFF030NEU.bed
EFO:0002784/ENCFF460TXJ.bed
CL:0000192/ENCFF343TOI.bed
EFO:0003042/ENCFF044YAN.bed
EFO:0006711/ENCFF253MNA.bed
EFO:0005719/ENCFF025UVM.bed
EFO:0007950/ENCFF402PGG.bed
UBERON:0000945/ENCFF516FFC.bed
UBERON:0001114/ENCFF610QGM.bed
NTR:0000711/ENCFF059ZLL.bed
UBERON:0001150/ENCFF336SKY.bed
UBERON:0001157/ENCFF080KPM.bed
UBERON:0000948/ENCFF045PUG.bed
UBERON:0001255/ENCFF436LXN.bed
UBERON:0000992/ENCFF827HSO.bed
UBERON:0001323/ENCFF340BKI.bed
UBERON:0001159/ENCFF296GNA.bed
UBERON:0001211/ENCFF439TOT.bed
UBE

In [35]:
exp_count = pd.read_csv("../all_marks/" + "H3K27me3" + "/narrow_peaks_signal_matrix.csv", sep="\t", index_col=0).shape[1]
ctl = int(exp_count/2)
exp = exp_count - ctl

In [61]:
df = pd.read_csv("../all_marks/" + "H3K27me3" + "/narrow_2_peaks_signal_matrix.csv", sep="\t", index_col=0)

In [63]:
df.max()

CL:0000062/ENCFF318IMH.bam          3004
CL:0000103/ENCFF122QYC.bed        115215
CL:0000127/ENCFF442ZRV.bed          4907
CL:0000182/ENCFF208KVX.bed         12569
CL:0000192/ENCFF327WQJ.bed         18459
CL:0000515/ENCFF232HUV.bed          2468
CL:0000746/ENCFF083APJ.bed         56910
CL:0000842/ENCFF193OFZ.bed         12105
CL:0002327/ENCFF015OVW.bed          2072
CL:0002328/ENCFF425HDW.bed          8113
CL:0002372/ENCFF287IGX.bed         12796
CL:0002518/ENCFF575FWP.bed          4168
CL:0002553/ENCFF535LEC.bed          5054
CL:1001606/ENCFF020YIZ.bed        126944
EFO:0001187/ENCFF030HAG.bed         3814
EFO:0001196/ENCFF825OWT.bed        14278
EFO:0002067/ENCFF190OWE.bed         3900
EFO:0002074/ENCFF127CPB.bed        10372
EFO:0002784/ENCFF175YYN.bed         1881
EFO:0003042/ENCFF083QQZ.bed        24466
EFO:0005719/ENCFF813CKH.bed         4279
EFO:0005723/ENCFF122SGX.bed         6559
EFO:0005904/ENCFF250WQM.bed        26059
EFO:0007950/ENCFF371CJE.bed       108127
NTR:0000711/ENCF